# Secondary Market Research Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/secondary_market_research_agents.ipynb)

This notebook demonstrates how to create a comprehensive secondary market research system using PraisonAI agents. The system generates detailed reports with customizable parameters for company, geography, and industry analysis.

## Features

- **Multi-agent Research System**: Specialized agents for different research areas
- **Customizable Parameters**: Company, geography, industry, and research sections
- **Comprehensive Reports**: Market overview, competitive analysis, financial performance, growth opportunities, and risk assessment
- **FastAPI Integration**: Ready for production deployment
- **Structured Output**: Professional report format suitable for business decision-making

## Installation

In [ ]:
# Install required dependencies
%pip install praisonai[crewai] > /dev/null
%pip install fastapi uvicorn > /dev/null
%pip install requests > /dev/null

## Setup API Key

In [ ]:
import os
from google.colab import userdata

# Set up OpenAI API key (add to Google Colab secrets)
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

## Method 1: Using YAML Configuration

In [ ]:
# YAML configuration for market research agents
agent_yaml = """
framework: "crewai"
topic: "Secondary Market Research Analysis"

# Configuration variables
variables:
  company: "Tesla"
  geography: "North America" 
  industry: "Electric Vehicles"

roles:
  market_overview_specialist:
    role: "Market Overview Specialist" 
    backstory: |
      You are a seasoned market research analyst with 10+ years of experience 
      in analyzing market dynamics, trends, and growth patterns.
    goal: |
      Analyze the {industry} market in {geography} to provide comprehensive 
      market overview including size, trends, and growth drivers.
    verbose: true
    allow_delegation: false
    tools:
      - "InternetSearchTool"
    tasks:
      market_overview_research:
        description: |
          Conduct comprehensive market overview research for {industry} in {geography}:
          1. Market size and valuation analysis
          2. Key growth trends and patterns
          3. Primary market drivers
          4. Market segmentation
        expected_output: |
          A comprehensive market overview report with quantitative data and insights.

  competitive_intelligence_analyst:
    role: "Competitive Intelligence Analyst"
    backstory: |
      You are a competitive intelligence expert specializing in analyzing 
      competitive landscapes and market positioning.
    goal: |
      Analyze {company}'s competitive landscape in the {industry} market.
    verbose: true
    allow_delegation: false
    tools:
      - "InternetSearchTool"
    tasks:
      competitive_analysis:
        description: |
          Conduct detailed competitive analysis for {company}:
          1. Identify top 5-7 competitors
          2. Market share analysis
          3. Competitive positioning
          4. Strengths and weaknesses
        expected_output: |
          A comprehensive competitive intelligence report.

  financial_performance_analyst:
    role: "Financial Performance Analyst"
    backstory: |
      You are a financial analyst with expertise in corporate finance 
      and industry benchmarking.
    goal: |
      Analyze {company}'s financial performance and industry benchmarks.
    verbose: true
    allow_delegation: false
    tools:
      - "InternetSearchTool"
    tasks:
      financial_analysis:
        description: |
          Conduct financial performance analysis for {company}:
          1. Revenue trends and growth
          2. Profitability metrics
          3. Financial ratios
          4. Industry benchmarking
        expected_output: |
          A detailed financial performance analysis report.

  research_report_synthesizer:
    role: "Research Report Synthesizer"
    backstory: |
      You are an expert business report writer specializing in 
      synthesizing complex research into actionable insights.
    goal: |
      Create a comprehensive secondary market research report for {company}.
    verbose: true
    allow_delegation: false
    tasks:
      report_synthesis:
        description: |
          Synthesize all research findings into a comprehensive report:
          1. Executive Summary
          2. Market Overview
          3. Competitive Analysis
          4. Financial Performance
          5. Conclusions and Recommendations
        expected_output: |
          A complete, professionally formatted secondary market research report.
        context:
          - "market_overview_research"
          - "competitive_analysis" 
          - "financial_analysis"

dependencies:
  - market_overview_specialist
  - competitive_intelligence_analyst
  - financial_performance_analyst
  - research_report_synthesizer
"""

In [ ]:
# Import required libraries
from praisonai import PraisonAI
from praisonai_tools import BaseTool
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# Create internet search tool
class InternetSearchTool(BaseTool):
    name: str = "InternetSearchTool"
    description: str = "Search Internet for relevant information"

    def _run(self, query: str):
        ddgs = DDGS()
        results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=5)
        return results

# Create PraisonAI instance
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[InternetSearchTool])

# Run the market research
print("🔍 Starting Secondary Market Research for Tesla...")
result = praisonai.run()

print("\n📋 Market Research Complete!")
print("=" * 60)
print(result)

## Method 2: Using Python API with Custom Parameters

In [ ]:
# Import the market research system
from praisonaiagents import Agent, Task, PraisonAIAgents, Tools
from typing import Dict, List, Any
import asyncio
from datetime import datetime

class MarketResearchConfig:
    """Configuration for market research parameters"""
    
    def __init__(self, company: str, geography: str, industry: str, sections: List[str]):
        self.company = company
        self.geography = geography
        self.industry = industry
        self.sections = sections
        self.timestamp = datetime.now().isoformat()

def create_market_research_agents(config: MarketResearchConfig):
    """Create specialized market research agents"""
    
    agents = {}
    
    if "market_overview" in config.sections:
        agents["market_overview"] = Agent(
            name="Market Overview Specialist",
            role="Market Analysis Expert",
            goal=f"Analyze the {config.industry} market in {config.geography}",
            instructions=f"""
            Analyze market size, trends, growth drivers for {config.industry} in {config.geography}.
            Provide quantitative data and cite reliable sources.
            """,
            tools=[Tools.internet_search],
            verbose=True
        )
    
    if "competitive_analysis" in config.sections:
        agents["competitive"] = Agent(
            name="Competitive Intelligence Analyst",
            role="Competition Research Expert", 
            goal=f"Analyze {config.company}'s competitive landscape",
            instructions=f"""
            Analyze {config.company}'s competitive position in {config.industry}.
            Identify key competitors, market share, and competitive advantages.
            """,
            tools=[Tools.internet_search],
            verbose=True
        )
    
    if "financial_performance" in config.sections:
        agents["financial"] = Agent(
            name="Financial Performance Analyst", 
            role="Financial Research Expert",
            goal=f"Analyze {config.company}'s financial performance",
            instructions=f"""
            Research {config.company}'s financial performance, revenue trends,
            profitability, and compare with industry benchmarks.
            """,
            tools=[Tools.internet_search],
            verbose=True
        )
    
    if "growth_opportunities" in config.sections:
        agents["growth"] = Agent(
            name="Growth Opportunities Researcher",
            role="Strategic Growth Expert",
            goal=f"Identify growth opportunities for {config.company}",
            instructions=f"""
            Identify emerging opportunities, market gaps, expansion possibilities
            for {config.company} in {config.industry}.
            """,
            tools=[Tools.internet_search],
            verbose=True
        )
    
    # Always include synthesizer
    agents["synthesizer"] = Agent(
        name="Research Report Synthesizer",
        role="Report Writing Expert",
        goal="Synthesize research into comprehensive report",
        instructions=f"""
        Create a professional secondary market research report for {config.company} 
        in {config.industry} with clear structure and actionable recommendations.
        """,
        verbose=True
    )
    
    return agents

def create_research_tasks(agents: Dict[str, Agent], config: MarketResearchConfig):
    """Create research tasks for the agents"""
    
    tasks = []
    
    if "market_overview" in config.sections and "market_overview" in agents:
        market_task = Task(
            name="market_overview_research",
            description=f"Research {config.industry} market overview in {config.geography}",
            expected_output="Market overview analysis with key metrics and trends",
            agent=agents["market_overview"]
        )
        tasks.append(market_task)
    
    if "competitive_analysis" in config.sections and "competitive" in agents:
        competitive_task = Task(
            name="competitive_analysis",
            description=f"Analyze competitive landscape for {config.company}",
            expected_output="Competitive analysis with key competitors and positioning",
            agent=agents["competitive"]
        )
        tasks.append(competitive_task)
    
    if "financial_performance" in config.sections and "financial" in agents:
        financial_task = Task(
            name="financial_analysis",
            description=f"Analyze {config.company}'s financial performance",
            expected_output="Financial performance analysis with key metrics",
            agent=agents["financial"]
        )
        tasks.append(financial_task)
    
    if "growth_opportunities" in config.sections and "growth" in agents:
        growth_task = Task(
            name="growth_opportunities",
            description=f"Identify growth opportunities for {config.company}",
            expected_output="Growth opportunities with strategic recommendations",
            agent=agents["growth"]
        )
        tasks.append(growth_task)
    
    # Synthesis task
    synthesis_task = Task(
        name="synthesis",
        description="Synthesize all findings into comprehensive report",
        expected_output="Complete secondary market research report",
        agent=agents["synthesizer"],
        context=tasks
    )
    tasks.append(synthesis_task)
    
    return tasks

async def run_market_research(config: MarketResearchConfig):
    """Run the complete market research workflow"""
    
    print(f"🔍 Starting Market Research for {config.company}")
    print(f"📍 Geography: {config.geography}")
    print(f"🏭 Industry: {config.industry}")
    print(f"📊 Sections: {', '.join(config.sections)}")
    print("=" * 60)
    
    # Create agents and tasks
    agents = create_market_research_agents(config)
    tasks = create_research_tasks(agents, config)
    
    # Create workflow
    workflow = PraisonAIAgents(
        agents=list(agents.values()),
        tasks=tasks,
        process="workflow",
        verbose=True
    )
    
    # Execute research
    results = await workflow.astart()
    
    return results

# Example with custom parameters
custom_config = MarketResearchConfig(
    company="Apple",
    geography="Global", 
    industry="Consumer Electronics",
    sections=["market_overview", "competitive_analysis", "financial_performance"]
)

# Run the research
print("🚀 Running Custom Market Research...")
custom_results = await run_market_research(custom_config)

print("\n📋 Research Complete!")
print("=" * 60)

## Method 3: FastAPI Integration Example

In [ ]:
# Example of using the FastAPI endpoint
import requests
import json
import time

# Note: This requires the FastAPI server to be running
# Run: uvicorn secondary-market-research-api:app --reload --port 8000

API_BASE_URL = "http://localhost:8000"  # Change if running elsewhere

def generate_report_via_api(company, geography, industry, sections):
    """Generate market research report via API"""
    
    # Prepare request
    request_data = {
        "company": company,
        "geography": geography,
        "industry": industry,
        "sections": sections,
        "format": "json"
    }
    
    print(f"📤 Submitting research request for {company}...")
    
    # Submit research request
    response = requests.post(f"{API_BASE_URL}/research/generate", json=request_data)
    
    if response.status_code == 200:
        job_data = response.json()
        job_id = job_data["job_id"]
        print(f"✅ Job submitted successfully. Job ID: {job_id}")
        
        # Poll for completion
        print("⏳ Waiting for completion...")
        while True:
            status_response = requests.get(f"{API_BASE_URL}/research/status/{job_id}")
            if status_response.status_code == 200:
                status_data = status_response.json()
                print(f"📊 Progress: {status_data['progress']}% - {status_data['message']}")
                
                if status_data["status"] == "completed":
                    print("🎉 Research completed!")
                    
                    # Download report
                    report_response = requests.get(f"{API_BASE_URL}/research/reports/{job_id}")
                    if report_response.status_code == 200:
                        return report_response.json()
                    else:
                        print("❌ Failed to download report")
                        return None
                        
                elif status_data["status"] == "failed":
                    print(f"❌ Research failed: {status_data.get('error', 'Unknown error')}")
                    return None
                    
                time.sleep(5)  # Wait 5 seconds before checking again
            else:
                print("❌ Failed to check status")
                return None
    else:
        print(f"❌ Failed to submit request: {response.status_code}")
        return None

# Example API usage (uncomment to run with API server)
# report = generate_report_via_api(
#     company="Microsoft",
#     geography="Global",
#     industry="Cloud Computing",
#     sections=["market_overview", "competitive_analysis"]
# )
# 
# if report:
#     print("\n📋 Generated Report:")
#     print(json.dumps(report, indent=2)[:1000] + "...")

print("💡 API integration example ready. Start the FastAPI server to test.")

## Customization Examples

In [ ]:
# Example configurations for different industries and regions

research_examples = {
    "tech_startup": {
        "company": "OpenAI",
        "geography": "United States",
        "industry": "Artificial Intelligence",
        "sections": ["market_overview", "competitive_analysis", "growth_opportunities"]
    },
    "automotive": {
        "company": "BMW",
        "geography": "Europe",
        "industry": "Luxury Automobiles",
        "sections": ["market_overview", "competitive_analysis", "financial_performance", "risk_assessment"]
    },
    "healthcare": {
        "company": "Pfizer",
        "geography": "Global",
        "industry": "Pharmaceuticals",
        "sections": ["market_overview", "competitive_analysis", "growth_opportunities", "risk_assessment"]
    },
    "fintech": {
        "company": "Square",
        "geography": "North America",
        "industry": "Financial Technology",
        "sections": ["market_overview", "competitive_analysis", "financial_performance", "growth_opportunities"]
    },
    "retail": {
        "company": "Alibaba",
        "geography": "Asia Pacific",
        "industry": "E-commerce",
        "sections": ["market_overview", "competitive_analysis", "financial_performance", "growth_opportunities", "risk_assessment"]
    }
}

print("📚 Research Configuration Examples:")
print("=" * 50)

for example_name, config in research_examples.items():
    print(f"\n🔍 {example_name.upper()}:")
    print(f"   Company: {config['company']}")
    print(f"   Geography: {config['geography']}")
    print(f"   Industry: {config['industry']}")
    print(f"   Sections: {', '.join(config['sections'])}")

print("\n💡 Use any of these configurations as templates for your research!")

## Summary

This notebook demonstrates three different ways to use the Secondary Market Research Agent system:

1. **YAML Configuration**: Easy to customize and modify research parameters
2. **Python API**: Programmatic control with custom configurations
3. **FastAPI Integration**: Production-ready REST API for web applications

### Key Features:
- **Customizable Research Sections**: Choose from market overview, competitive analysis, financial performance, growth opportunities, and risk assessment
- **Geographic Flexibility**: Research any geographic region
- **Industry Agnostic**: Works across different industries and sectors
- **Professional Reports**: Generate business-ready research reports
- **Scalable Architecture**: Multi-agent system that can be extended

### Use Cases:
- Market entry analysis
- Competitive intelligence
- Investment research
- Strategic planning
- Due diligence

The system provides a comprehensive foundation for secondary market research that can be customized and extended based on specific business needs.